In [1]:
import pandas as pd, numpy as np
import os, time, gc
from tqdm import tqdm
pd.options.mode.chained_assignment = None

In [2]:
os.listdir('input')

['.ipynb_checkpoints',
 'new_te.feather',
 'new_te_abl.feather',
 'new_te_cam.feather',
 'new_te_rc.feather',
 'new_tr.feather',
 'new_tr_abl.feather',
 'new_tr_cam.feather',
 'new_tr_rc.feather',
 'sample_submission.csv',
 'te.feather',
 'test.csv',
 'te_abl.feather',
 'te_cam.feather',
 'te_rc.feather',
 'te_v7.feather',
 'tr.feather',
 'train.csv',
 'tr_abl.feather',
 'tr_rc.feather',
 'tr_v7.feather']

In [3]:
%%time
tr = pd.read_feather('../dacon_sc/input/tr.feather')
te = pd.read_feather('../dacon_sc/input/te.feather')
submission = pd.read_csv('../dacon_sc/input/sample_submission.csv')

Wall time: 34.1 s


In [4]:
tr.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [5]:
def make_timespan(df):

    df['time_span'] = 0
    df['time_span'] = np.where(df['time']<1, 0, 
                                  np.where(df['time']<2, 1, 
                                          np.where(df['time']<3, 2,
                                                  np.where(df['time']<4, 3,
                                                          np.where(df['time']<5, 4,
                                                                  np.where(df['time']<6, 5,
                                                                          np.where(df['time']<7, 6,
                                                                                  np.where(df['time']<8, 7,
                                                                                          np.where(df['time']<9, 8,
                                                                                                   np.where(df['time']<10, 9, 10))))))))))
    return df

In [6]:
tr.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


# Event

In [7]:
tr['game_id'].unique()

array([    0,     1,     2, ..., 38869, 38870, 38871], dtype=int64)

In [8]:
tr['event'].unique().tolist()

['Camera',
 'Selection',
 'Ability',
 'Right Click',
 'SetControlGroup',
 'GetControlGroup',
 'AddToControlGroup',
 'ControlGroup']

In [9]:
events = tr['event'].unique().tolist()

def event_timespan(df):
    # make timespan
    df = make_timespan(df)
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p0 = df[df['player']==0]
        tmp = p0[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=True).unstack()
        tmp.columns = ['p0_tsp_{}_'.format(eve_name)+str(i) for i in range(0,11)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
        del tmp;gc.collect()
    
    del p0;gc.collect()
    for eve_name in tqdm(events, total=len(events)):
        p1 = df[df['player']==1]
        tmp = p1[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=True).unstack()
        tmp.columns = ['p1_tsp_{}_'.format(eve_name)+str(i) for i in range(0,11)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df

In [14]:
new_tr_eve = event_timespan(tr)
new_te_eve = event_timespan(te)
print(new_tr_eve.shape, new_te_eve.shape)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]c:\users\roal2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:30<00:00,  3.76s/it]


(38872, 177) (16787, 177)


In [15]:
new_tr_eve.head()

,game_id,p0_tsp_Camera_0,p0_tsp_Camera_1,p0_tsp_Camera_2,p0_tsp_Camera_3,p0_tsp_Camera_4,p0_tsp_Camera_5,p0_tsp_Camera_6,p0_tsp_Camera_7,p0_tsp_Camera_8,...,p1_tsp_ControlGroup_1,p1_tsp_ControlGroup_2,p1_tsp_ControlGroup_3,p1_tsp_ControlGroup_4,p1_tsp_ControlGroup_5,p1_tsp_ControlGroup_6,p1_tsp_ControlGroup_7,p1_tsp_ControlGroup_8,p1_tsp_ControlGroup_9,p1_tsp_ControlGroup_10
0,0,0.101351,0.083333,0.157658,0.171171,0.137387,0.135135,0.144144,0.069820,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.031898,0.062201,0.143541,0.065391,0.094099,0.121212,0.097289,0.119617,0.141946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.116223,0.055690,0.048426,0.106538,0.121065,0.094431,0.096852,0.072639,0.215496,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,0.037868,0.060309,0.099579,0.088359,0.096774,0.145863,0.091164,0.123422,0.088359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.069767,0.160465,0.237209,0.190698,0.167442,0.174419,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
new_te_eve.head()

,game_id,p0_tsp_Camera_0,p0_tsp_Camera_1,p0_tsp_Camera_2,p0_tsp_Camera_3,p0_tsp_Camera_4,p0_tsp_Camera_5,p0_tsp_Camera_6,p0_tsp_Camera_7,p0_tsp_Camera_8,...,p1_tsp_ControlGroup_1,p1_tsp_ControlGroup_2,p1_tsp_ControlGroup_3,p1_tsp_ControlGroup_4,p1_tsp_ControlGroup_5,p1_tsp_ControlGroup_6,p1_tsp_ControlGroup_7,p1_tsp_ControlGroup_8,p1_tsp_ControlGroup_9,p1_tsp_ControlGroup_10
0,38872,0.081897,0.021552,0.198276,0.120690,0.168103,0.198276,0.181034,0.030172,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38873,0.185000,0.070000,0.060000,0.190000,0.305000,0.165000,0.025000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38874,0.228571,0.224490,0.208163,0.200000,0.138776,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38875,0.073786,0.064078,0.172816,0.097087,0.174757,0.157282,0.149515,0.110680,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38876,0.558140,0.294574,0.147287,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
new_tr_eve.to_feather('../dacon_sc/input/new_tr_eve_ts.feather')
new_te_eve.to_feather('../dacon_sc/input/new_te_eve_ts.feather')

# Camera

In [18]:
def cam_feat(df):

    cam = df[df['event']=="Camera"]
    
    cam['cam_lat'] = cam['event_contents'].apply(lambda x: x.split("(")[1].split(",")[0])
    cam['cam_lon'] = cam['event_contents'].apply(lambda x: x.split("(")[1].split(",")[1].split(" ")[1].split(")")[0])

    cam['cam_lat'] = cam['cam_lat'].astype(float)
    cam['cam_lon'] = cam['cam_lon'].astype(float)

    cam['cam_lat'] = cam['cam_lat'].apply(lambda x: round(x, 6))
    cam['cam_lon'] = cam['cam_lon'].apply(lambda x: round(x, 6))
    
    return cam 

In [19]:
%%time
tr_cam = cam_feat(tr)
tr_cam = make_timespan(tr_cam)

Wall time: 1min 54s


In [20]:
%%time
# tr_cam = pd.read_feather('../dacon_sc/input/tr_cam.feather')
tr_rc = pd.read_feather('../dacon_sc/input/tr_rc.feather')
tr_abl = pd.read_feather('../dacon_sc/input/tr_abl.feather')

Wall time: 11.5 s


In [21]:
tr_rc['rclick_lat'] = tr_rc['rclick_lat'].astype(float)
tr_rc['rclick_lon'] = tr_rc['rclick_lon'].astype(float)
tr_rc['rclick_height'] = tr_rc['rclick_height'].astype(int)

In [28]:
%%time
te_cam = pd.read_feather('../dacon_sc/input/te_cam.feather')
te_rc  = pd.read_feather('../dacon_sc/input/te_rc.feather')
te_abl = pd.read_feather('../dacon_sc/input/te_abl.feather')
te_rc['rclick_lat'] = te_rc['rclick_lat'].astype(float)
te_rc['rclick_lon'] = te_rc['rclick_lon'].astype(float)
te_rc['rclick_height'] = te_rc['rclick_height'].astype(int)

te_cam = make_timespan(te_cam)

In [23]:
# tr_cam['cam_tsp'] = tr_cam['event']+"_"+tr_cam['time_span'].astype(str)

In [24]:
%%time
tr_cam['cam_lat_r2'] = tr_cam['cam_lat'].apply(lambda x: round(x, 2))
tr_cam['cam_lon_r2'] = tr_cam['cam_lon'].apply(lambda x: round(x, 2))
tr_cam['cam_lat_int'] = tr_cam['cam_lat'].apply(lambda x: round(x, 0))
tr_cam['cam_lon_int'] = tr_cam['cam_lon'].apply(lambda x: round(x, 0))

te_cam['cam_lat_r2'] = te_cam['cam_lat'].apply(lambda x: round(x, 2))
te_cam['cam_lon_r2'] = te_cam['cam_lon'].apply(lambda x: round(x, 2))
te_cam['cam_lat_int'] = te_cam['cam_lat'].apply(lambda x: round(x, 0))
te_cam['cam_lon_int'] = te_cam['cam_lon'].apply(lambda x: round(x, 0))

Wall time: 1min 57s


In [25]:
tr_cam.head()

,game_id,winner,time,player,species,event,event_contents,time_span,cam_lat,cam_lon,cam_lat_r2,cam_lon_r2,cam_lat_int,cam_lon_int
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",0,145.250000,21.507812,145.25,21.51,145.0,22.0
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)",0,22.750000,147.007812,22.75,147.01,23.0,147.0
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",0,142.996094,24.503906,143.00,24.50,143.0,25.0
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)",0,142.507812,24.988281,142.51,24.99,143.0,25.0
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)",0,139.617188,27.882812,139.62,27.88,140.0,28.0


In [26]:
def cam_tsp(df):
    
    p0 = df[df['player']==0]
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    
    for col_name in ['cam_lat_r2','cam_lon_r2']:
        for tsp in tqdm(range(11)):
            tmp = p0[p0['time_span']==tsp].groupby(['game_id','time_span'])[col_name].agg(['mean','std']).unstack()
            tmp.columns = ['p0_{}_tsp_{}_mean'.format(col_name,tsp),'p0_{}_tsp_{}_std'.format(col_name,tsp)]
            tmp.reset_index(inplace=True)
            new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
    del p0, tmp;gc.collect()
    
    p1 = df[df['player']==1]
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    
    for col_name in ['cam_lat_r2','cam_lon_r2']:
        for tsp in tqdm(range(11)):
            tmp = p1[p1['time_span']==tsp].groupby(['game_id','time_span'])[col_name].agg(['mean','std']).unstack()
            tmp.columns = ['p1_{}_tsp_{}_mean'.format(col_name,tsp),'p1_{}_tsp_{}_std'.format(col_name,tsp)]
            tmp.reset_index(inplace=True)
            new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1, tmp;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df    

In [29]:
new_tr_cam = cam_tsp(tr_cam)
new_te_cam = cam_tsp(te_cam)
print(new_tr_cam.shape, new_te_cam.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  5.84it/s]


(38872, 89) (16787, 89)


In [30]:
new_tr_cam.head()

,game_id,p0_cam_lat_r2_tsp_0_mean,p0_cam_lat_r2_tsp_0_std,p0_cam_lat_r2_tsp_1_mean,p0_cam_lat_r2_tsp_1_std,p0_cam_lat_r2_tsp_2_mean,p0_cam_lat_r2_tsp_2_std,p0_cam_lat_r2_tsp_3_mean,p0_cam_lat_r2_tsp_3_std,p0_cam_lat_r2_tsp_4_mean,...,p1_cam_lon_r2_tsp_6_mean,p1_cam_lon_r2_tsp_6_std,p1_cam_lon_r2_tsp_7_mean,p1_cam_lon_r2_tsp_7_std,p1_cam_lon_r2_tsp_8_mean,p1_cam_lon_r2_tsp_8_std,p1_cam_lon_r2_tsp_9_mean,p1_cam_lon_r2_tsp_9_std,p1_cam_lon_r2_tsp_10_mean,p1_cam_lon_r2_tsp_10_std
0,0,138.240000,4.327896,142.152973,2.428270,141.559429,1.778345,134.327105,27.850528,90.235410,...,132.602187,8.459990,135.447143,7.685361,NaN,NaN,NaN,NaN,NaN,NaN
1,1,136.736000,3.840295,138.480256,0.313558,137.840111,4.173548,124.927317,24.240364,136.044746,...,73.475814,58.448197,65.282688,50.843657,77.214865,42.734953,38.236864,28.268899,NaN,NaN
2,2,142.365833,10.150102,103.663043,56.213800,146.966000,0.221155,147.110000,0.198225,143.595800,...,102.553780,14.385992,94.582174,24.813243,95.050778,21.274845,98.345422,18.510057,NaN,NaN
3,3,123.653333,3.803361,121.568837,5.092867,120.315493,19.476737,57.232857,34.287014,79.240435,...,121.547273,17.017540,116.663768,25.222990,86.342600,32.138820,63.322063,24.809383,NaN,NaN
4,4,50.313667,27.124572,126.203913,51.683153,71.105686,34.312824,56.979878,12.659994,50.682361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
new_te_cam.head()

,game_id,p0_cam_lat_r2_tsp_0_mean,p0_cam_lat_r2_tsp_0_std,p0_cam_lat_r2_tsp_1_mean,p0_cam_lat_r2_tsp_1_std,p0_cam_lat_r2_tsp_2_mean,p0_cam_lat_r2_tsp_2_std,p0_cam_lat_r2_tsp_3_mean,p0_cam_lat_r2_tsp_3_std,p0_cam_lat_r2_tsp_4_mean,...,p1_cam_lon_r2_tsp_6_mean,p1_cam_lon_r2_tsp_6_std,p1_cam_lon_r2_tsp_7_mean,p1_cam_lon_r2_tsp_7_std,p1_cam_lon_r2_tsp_8_mean,p1_cam_lon_r2_tsp_8_std,p1_cam_lon_r2_tsp_9_mean,p1_cam_lon_r2_tsp_9_std,p1_cam_lon_r2_tsp_10_mean,p1_cam_lon_r2_tsp_10_std
0,38872,30.351053,3.957294,69.394000,58.391946,30.823261,18.163733,32.492143,4.653681,26.615897,...,138.468026,11.720684,120.390769,4.406793,NaN,NaN,NaN,NaN,NaN,NaN
1,38873,48.129189,46.019020,29.722143,8.824885,55.030000,26.559259,64.397368,50.136090,26.217377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38874,150.206250,0.738073,96.575455,53.954167,102.174706,56.315303,143.013061,11.042534,77.759118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38875,150.998684,3.041020,147.526970,4.119535,130.089888,40.744987,132.814400,35.908798,95.844333,...,36.131000,9.263225,34.016296,9.121251,NaN,NaN,NaN,NaN,NaN,NaN
4,38876,120.594306,19.054475,111.287895,23.408719,127.608947,4.931778,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
new_tr_cam.to_feather('../dacon_sc/input/new_tr_cam_ts.feather')
new_te_cam.to_feather('../dacon_sc/input/new_te_cam_ts.feather')

# Right Click

In [33]:
tr_rc.head()

,game_id,winner,time,player,species,event,event_contents,rclick_act,rclick_obj,rclick_lat,rclick_lon,rclick_height
0,0,1,0.20,0,T,Right Click,"Location: (136.3759765625, 41.11962890625, 49016)",Location,None,136.375977,41.119629,49016
1,0,1,0.23,0,T,Right Click,"Location: (136.033935546875, 41.735107421875, ...",Location,None,136.033936,41.735107,49104
2,0,1,1.17,0,T,Right Click,"Location: (32.222900390625, 153.720703125, 49120)",Location,None,32.222900,153.720703,49120
3,0,1,1.18,0,T,Right Click,"Location: (29.195068359375, 143.441650390625, ...",Location,None,29.195068,143.441650,49120
4,0,1,1.18,0,T,Right Click,"Location: (29.195068359375, 129.53466796875, 4...",Location,None,29.195068,129.534668,45828


In [34]:
%%time
tr_rc['rc_lat_r2'] = tr_rc['rclick_lat'].apply(lambda x: round(x, 2))
tr_rc['rc_lon_r2'] = tr_rc['rclick_lon'].apply(lambda x: round(x, 2))
tr_rc['rc_lat_int'] = tr_rc['rclick_lat'].apply(lambda x: round(x, 0))
tr_rc['rc_lon_int'] = tr_rc['rclick_lon'].apply(lambda x: round(x, 0))

te_rc['rc_lat_r2'] = te_rc['rclick_lat'].apply(lambda x: round(x, 2))
te_rc['rc_lon_r2'] = te_rc['rclick_lon'].apply(lambda x: round(x, 2))
te_rc['rc_lat_int'] = te_rc['rclick_lat'].apply(lambda x: round(x, 0))
te_rc['rc_lon_int'] = te_rc['rclick_lon'].apply(lambda x: round(x, 0))

Wall time: 42.9 s


In [35]:
tr_rc = make_timespan(tr_rc)
te_rc = make_timespan(te_rc)

In [36]:
def rc_tsp(df):
    
    p0 = df[df['player']==0]
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    
    for col_name in ['rc_lat_r2' ,	'rc_lon_r2' ,	'rclick_height']:
        for tsp in tqdm(range(11)):
            tmp = p0[p0['time_span']==tsp].groupby(['game_id','time_span'])[col_name].agg(['mean','std']).unstack()
            tmp.columns = ['p0_{}_tsp_{}_mean'.format(col_name,tsp),'p0_{}_tsp_{}_std'.format(col_name,tsp)]
            tmp.reset_index(inplace=True)
            new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
    del p0, tmp;gc.collect()
    
    p1 = df[df['player']==1]
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    
    for col_name in ['rc_lat_r2' ,	'rc_lon_r2' ,	'rclick_height']:
        for tsp in tqdm(range(11)):
            tmp = p1[p1['time_span']==tsp].groupby(['game_id','time_span'])[col_name].agg(['mean','std']).unstack()
            tmp.columns = ['p1_{}_tsp_{}_mean'.format(col_name,tsp),'p1_{}_tsp_{}_std'.format(col_name,tsp)]
            tmp.reset_index(inplace=True)
            new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1, tmp;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df    

In [37]:
new_tr_rc = rc_tsp(tr_rc)
new_te_rc = rc_tsp(te_rc)
print(new_tr_rc.shape, new_te_rc.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 11.52it/s]


(38869, 133) (16786, 133)


In [38]:
new_tr_rc.head()

,game_id,p0_rc_lat_r2_tsp_0_mean,p0_rc_lat_r2_tsp_0_std,p0_rc_lat_r2_tsp_1_mean,p0_rc_lat_r2_tsp_1_std,p0_rc_lat_r2_tsp_2_mean,p0_rc_lat_r2_tsp_2_std,p0_rc_lat_r2_tsp_3_mean,p0_rc_lat_r2_tsp_3_std,p0_rc_lat_r2_tsp_4_mean,...,p1_rclick_height_tsp_6_mean,p1_rclick_height_tsp_6_std,p1_rclick_height_tsp_7_mean,p1_rclick_height_tsp_7_std,p1_rclick_height_tsp_8_mean,p1_rclick_height_tsp_8_std,p1_rclick_height_tsp_9_mean,p1_rclick_height_tsp_9_std,p1_rclick_height_tsp_10_mean,p1_rclick_height_tsp_10_std
0,0,136.205000,0.247487,49.654286,45.621141,146.386667,7.080151,89.880000,67.288281,77.074444,...,46389.333333,4729.653405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,134.941111,9.147167,139.335556,1.258343,136.296190,9.508004,126.838235,20.921735,142.174118,...,45465.166667,4187.438600,42719.000000,6956.644737,37350.708333,5400.538512,43838.444444,3987.084276,NaN,NaN
2,2,145.190833,3.226043,77.887917,59.299739,152.162941,4.917082,157.250000,3.889087,146.854167,...,38922.923077,4982.956660,41846.111111,4924.470465,37576.794118,4103.192975,36688.375000,4347.968099,NaN,NaN
3,3,124.033929,6.419294,126.091818,6.863188,103.681429,37.537242,34.612787,24.406479,123.100526,...,41709.739130,3151.573867,44548.280000,4776.017837,41454.750000,2015.027079,41937.419355,3849.096030,NaN,NaN
4,4,53.525882,26.687911,147.981463,16.497218,69.422857,27.732112,75.787576,16.776450,51.840833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
new_te_rc.head()

,game_id,p0_rc_lat_r2_tsp_0_mean,p0_rc_lat_r2_tsp_0_std,p0_rc_lat_r2_tsp_1_mean,p0_rc_lat_r2_tsp_1_std,p0_rc_lat_r2_tsp_2_mean,p0_rc_lat_r2_tsp_2_std,p0_rc_lat_r2_tsp_3_mean,p0_rc_lat_r2_tsp_3_std,p0_rc_lat_r2_tsp_4_mean,...,p1_rclick_height_tsp_6_mean,p1_rclick_height_tsp_6_std,p1_rclick_height_tsp_7_mean,p1_rclick_height_tsp_7_std,p1_rclick_height_tsp_8_mean,p1_rclick_height_tsp_8_std,p1_rclick_height_tsp_9_mean,p1_rclick_height_tsp_9_std,p1_rclick_height_tsp_10_mean,p1_rclick_height_tsp_10_std
0,38872,44.076364,39.128015,38.433571,42.130169,34.345000,27.271873,36.593617,20.916705,28.666364,...,42982.769231,3547.469148,40933.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38873,112.731667,48.320141,24.013333,1.841204,18.940000,1.187603,38.437083,34.607335,21.740000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38874,137.556667,35.481030,60.018182,58.053180,56.261613,42.049184,112.916923,27.243946,63.672500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38875,152.202400,6.105891,131.481667,43.315732,130.373824,45.688879,118.683500,52.952517,54.594000,...,46297.918367,6978.773749,46446.733333,4286.51153,NaN,NaN,NaN,NaN,NaN,NaN
4,38876,107.786250,33.690174,83.604000,17.981766,125.200000,10.285410,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
new_tr_rc.to_feather('../dacon_sc/input/new_tr_rc_ts.feather')
new_te_rc.to_feather('../dacon_sc/input/new_te_rc_ts.feather')

# Ability

In [ ]:
# tr_abl['act_name'].nunique(), tr_abl['act_id'].nunique()

# tr_abl = make_timespan(tr_abl)

In [ ]:
# tr_abl.iloc[:,-4:].nunique()

In [ ]:
# def abl_tsp(df):
#     abil_acts = df.act_id.unique()
#     p0 = df[df['player']==0]
#     new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    
    
#     for act_id in tqdm(abil_acts, total=len(abil_acts)):
#         p0 = df[df['player']==0]
#         tmp = p0[df['act_id']==act_id].groupby(['game_id'])['time_span'].agg(['mean','std','count'])#.unstack()
#         tmp.columns = ['p0_{}_tsp_mean'.format(act_id),'p0_{}_tsp_std'.format(act_id),'p0_{}_tsp_cnt'.format(act_id)]
#         tmp.reset_index(inplace=True)
#         tmp.fillna(0, inplace=True)
#         new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
    
#     del tmp;gc.collect()
    
#     p1 = df[df['player']==1]
#     new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
#     for act_id in tqdm(abil_acts, total=len(abil_acts)):
#         p1 = df[df['player']==1]
#         tmp = p1[df['act_id']==act_id].groupby(['game_id'])['time_span'].agg(['mean','std','count'])#.unstack()
#         tmp.columns = ['p1_{}_tsp_mean'.format(act_id),'p1_{}_tsp_std'.format(act_id),'p1_{}_tsp_cnt'.format(act_id)]
#         tmp.reset_index(inplace=True)
#         tmp.fillna(0, inplace=True)
#         new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    
#     del tmp;gc.collect()
    
#     new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
#     return new_df    

In [ ]:
# new_tr_abl = abl_tsp(tr_abl)

In [ ]:
# new_tr_abl.head()